In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression
from sklearn.preprocessing import FunctionTransformer, KBinsDiscretizer, StandardScaler, MinMaxScaler, PolynomialFeatures, RobustScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.pipeline import make_pipeline
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import pickle


In [2]:
data_path = "Dataset_Brief.csv"
df = pd.read_csv(data_path)

In [3]:
df = df.dropna()

df['age'].fillna(df['age'].mean(), inplace=True)


In [4]:
z_scores = stats.zscore(df['bmi'])

df = df[(z_scores < 3) & (z_scores > -3)]
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [5]:
# df = pd.get_dummies(df, columns=['sex', 'smoker', 'region'], drop_first=True)
# df

In [6]:
X = df.drop('charges', axis=1)
y = df['charges']
X = pd.get_dummies(df, columns=['sex', 'smoker', 'region'], drop_first=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker_yes'])

In [7]:
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [8]:
r2_cleaned = r2_score(y_test, y_pred)
mse_cleaned = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse_cleaned = np.sqrt(mse_cleaned)

print("Performance du modèle après suppression des valeurs extrêmes:")
print(f"R2 Score: {r2_cleaned:.4f}")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse_cleaned:.2f}")

Performance du modèle après suppression des valeurs extrêmes:
R2 Score: 1.0000
MAE: 0.00
RMSE: 0.00


In [9]:
with open('modele.pkl', 'wb') as file:
    pickle.dump(grid_search, file)

NameError: name 'grid_search' is not defined